In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from pipeline import *
from data_utils import *
from utils import *
import vis
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.metrics import accuracy_score
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style

%matplotlib inline
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA

In [2]:
# Need to load checkpoint
checkpoint_id = "cnn_rnn_end_to_end_cifar100"
model = load_model(checkpoint_id, is_training=True, checkpoint_model_id=checkpoint_id)

Loading model...
../tensorboard_logs/cnn_rnn_end_to_end_cifar100/
../checkpoints/cnn_rnn_end_to_end_cifar100/
../best_checkpoints/cnn_rnn_end_to_end_cifar100/
Checkpoint loaded.
Model loaded.


In [3]:
# Load dataset
X_test, y_test, fine_or_coarse_test = load_data_pyramid(dataset='cifar100_joint_prefeaturized', return_subset='test_only')

In [ ]:
# Test on dataset
pred_test_probs = np.asarray(model.predict(X_test))

In [ ]:
pred_test_probs.shape
reshaped = pred_test_probs.reshape((-1, 2, 121))



In [ ]:
# Class predictions are in the form [A, B] (Coarse, Fine). 
def simplifyToHierarchicalFormat(class_predictions, end_token = 120):
    fine_labels_double_copy = np.reshape(np.tile(class_predictions, (2, 1)), (2, -1, 2))
    fine_labels_first = fine_labels_double_copy[0,:,1]
    fine_labels_second = fine_labels_double_copy[1,:,1]
    
    coarse_labels_mask = fine_labels_first
    coarse_labels_mask[coarse_labels_mask != end_token] = 0
    coarse_labels_mask[coarse_labels_mask == end_token] = 1
    
    hierarchical_labels =  fine_labels_second
    hierarchical_labels[hierarchical_labels == end_token] = 0
    
    return hierarchical_labels + class_predictions[:,0]*coarse_labels_mask
    
    
# Make predictions
pred_test_probs.shape
reshaped = pred_test_probs.reshape((-1, 2, 121))
class_predictions = np.argmax(reshaped, axis=2)

print class_predictions
print simplifyToHierarchicalFormat(class_predictions)


In [ ]:
# Print out examples
import pickle
p = pickle.load(open("coarse_to_fine_map.pickle"))